# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster_Response.db')
df = pd.read_sql_table('Disaster_messages', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # removing punctuations and normalizing the case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #creating word tokens
    tokens = word_tokenize(text)
    
    # transforming the words to their root form which are not in stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f58063d6f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# splitting the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# fitting training data on classifier
pipeline.fit(X_train, y_train)

# predicting on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# function to create classification report of all the categories of the dataset

def class_report(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))
    accuracy = round((y_pred == y_test.values).mean(), 3)
    print('The model accuracy is {}'.format(accuracy))

In [10]:
class_report(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.64      0.44      0.52      2043
          1       0.84      0.92      0.88      6547

avg / total       0.79      0.81      0.79      8590

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      7126
          1       0.80      0.44      0.56      1464

avg / total       0.88      0.89      0.87      8590

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8553
          1       0.00      0.00      0.00        37

avg / total       0.99      1.00      0.99      8590

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      4975
          1       0.75      0.60      0.67      3615

avg / total       0.75      0.75      0.74      8590

medical_help
             precision    recall  f1-score   support

          0       0.92      0.99      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [30, 60], 
              } 

cv = GridSearchCV(pipeline, param_grid=parameters)

In [12]:
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [30, 60]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [30, 60]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
class_report(cv, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.70      0.38      0.50      2043
          1       0.83      0.95      0.89      6547

avg / total       0.80      0.81      0.79      8590

request
             precision    recall  f1-score   support

          0       0.91      0.98      0.94      7126
          1       0.82      0.51      0.63      1464

avg / total       0.89      0.90      0.89      8590

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8553
          1       0.00      0.00      0.00        37

avg / total       0.99      1.00      0.99      8590

aid_related
             precision    recall  f1-score   support

          0       0.79      0.85      0.82      4975
          1       0.77      0.68      0.72      3615

avg / total       0.78      0.78      0.78      8590

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# to improve the model, added text length extractor transformer with feature union
pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [16]:
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f58063d6f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [17]:
# train the new model, predict and generate a classification report
pipeline_ada.fit(X_train, y_train)
class_report(pipeline_ada, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.68      0.18      0.28      2043
          1       0.79      0.97      0.87      6547

avg / total       0.76      0.78      0.73      8590

request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      7126
          1       0.75      0.51      0.61      1464

avg / total       0.88      0.89      0.88      8590

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8553
          1       0.00      0.00      0.00        37

avg / total       0.99      0.99      0.99      8590

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      4975
          1       0.75      0.61      0.67      3615

avg / total       0.75      0.75      0.74      8590

medical_help
             precision    recall  f1-score   support

          0       0.94      0.99      0

In [18]:
parameters_ada = {'tfidf__use_idf': (True, False),
                  'clf__estimator__n_estimators': [30, 60],
                  'clf__estimator__learning_rate': [1, 2],
                  } 

cv_ada = GridSearchCV(pipeline_ada, param_grid=parameters_ada)

In [19]:
cv_ada

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [30, 60], 'clf__estimator__learning_rate': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
cv_ada.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [30, 60], 'clf__estimator__learning_rate': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
class_report(cv_ada, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.68      0.21      0.32      2043
          1       0.80      0.97      0.87      6547

avg / total       0.77      0.79      0.74      8590

request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      7126
          1       0.74      0.53      0.62      1464

avg / total       0.88      0.89      0.88      8590

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8553
          1       0.00      0.00      0.00        37

avg / total       0.99      0.99      0.99      8590

aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      4975
          1       0.76      0.61      0.68      3615

avg / total       0.76      0.76      0.75      8590

medical_help
             precision    recall  f1-score   support

          0       0.94      0.99      0

### 9. Export your model as a pickle file

In [22]:
file_name = 'classifier.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.